**卡方分布**

卡方分布(chi-square distribution)是概率统计里常用的一种概率分布，也是统计推断里应该用最广泛的概率分布之一，在假设检验与置信区间的计算中经常能见到卡方分布。

卡方分布的定义如下：   
若k个独立的随机变量Z1,Z2,...,Zk满足标准正态分布N(0,1),则这k个随机变量的平方和：
![卡方公式1](./images/卡方分布公式1.png)   
为服从自由度为k的卡方分布，记作：   
![卡方公式2](./images/卡方公式2.png)   
或者记作   
![卡方公式3](./images/卡方公式3.png)

**卡方检验**

卡方检验是以卡方分布为基础的一种假设检验方法，主要用于分类变量之间的独立性检验。其基本思想是根据样本数据推断**总体的分布与期望分布是否有显著性差异**，或者**推断两个分类变量是否相关或者独立。** 
一般可以设原假设为：观察频数与期望频数没有差异，或者两个变量相互独立不相关。实际应用中，我们先假设原假设成立，计算出卡方的值，卡方表示观察值与理论值见得偏离程度。

**卡方的计算公式**

![卡方公式detail](./images/卡方公式detail.png)

其中Ai为i水平的观察频数，Ei为i水平的期望频数，n为总频数，pi为i水平的期望频率。i水平的期望频数Ei等于总频数n\*i水平的期望概率pi，k为单元格数。当n比较大时，χ2统计量近似服从k-1(计算Ei时用到的参数个数)个自由度的卡方分布。   
卡方值包含了以下两个信息：   
- 实际值与理论值偏差的绝对大小。
- 差异程度与理论值的相对大小。

上述计算的卡方值服从卡方分布。根据卡方分布，卡方统计量以及自由度，可以确定在原假设成立的情况下获得当前统计量以及更极端情况的概率p。如果p很小，说明观察值与理论指的偏离程度大，应该拒绝假设。否则不能拒绝原假设。

**卡方检验示例**

某医院对某种病症的患者使用了A、B两种不同的疗法，结果如下，问两种疗法有无差别？

In [4]:
import numpy as np
import pandas as pd
nd_data = np.array([[19,24,43,44.2],[34,10,44,77.3],[53,34,87,60.9]])
df = pd.DataFrame(nd_data,index=['A组','B组','合计'],
                  columns=['有效','无效','合计','有效率'])
df

,有效,无效,合计,有效率
A组,19.0,24.0,43.0,44.2
B组,34.0,10.0,44.0,77.3
合计,53.0,34.0,87.0,60.9


In [9]:
df['有效期望频数'] = np.nan
df['无效期望频数'] = np.nan
df

,有效,无效,合计,有效率,有效期望频数,无效期望频数
A组,19.0,24.0,43.0,44.2,NaN,NaN
B组,34.0,10.0,44.0,77.3,NaN,NaN
合计,53.0,34.0,87.0,60.9,NaN,NaN


In [13]:
df.loc['A组']['有效期望频数'] = round(43 * 53 / 87,1)
df.loc['A组']['无效期望频数'] = round(43 * 34 / 87,1)
df.loc['B组']['有效期望频数'] = round(44 * 53 / 87,1)
df.loc['B组']['无效期望频数'] = round(44 * 34 / 87,1)
df

,有效,无效,合计,有效率,有效期望频数,无效期望频数
A组,19.0,24.0,43.0,44.2,26.2,16.8
B组,34.0,10.0,44.0,77.3,26.8,17.2
合计,53.0,34.0,87.0,60.9,NaN,NaN


卡方值计算

In [15]:
ka_2 = pow(19-26.2,2)/26.2 + pow(34-26.8,2)/26.8\
+ pow(24-16.8,2)/16.8 + pow(10-17.2,2)/17.2
ka_2

10.012622086493804

得到卡方值以后，接下来需要查询卡方分布表来判断p值，从而做出接受或拒绝原假设的决定。   
首先我们明确自由度的概念：自由度k=(行数-1) * (列数-1)。这里k=1,然后看卡方分布的临界概率表，我们可以用如下代码生成：

In [42]:
from scipy.stats import chi2
import pandas as pd

# chi square distribution
percents = [0.95,0.90,0.5,0.1,0.05,0.025,0.01,0.005]

df = pd.DataFrame(np.array([chi2.isf(percents, i) for i in range(1,30)]),
                 index = np.arange(1,30),columns = percents)
pd.set_option('precision',3)
df.head()

,0.95,0.9,0.5,0.1,0.05,0.025,0.01,0.005
1,0.004,0.016,0.455,2.706,3.841,5.024,6.635,7.879
2,0.103,0.211,1.386,4.605,5.991,7.378,9.210,10.597
3,0.352,0.584,2.366,6.251,7.815,9.348,11.345,12.838
4,0.711,1.064,3.357,7.779,9.488,11.143,13.277,14.860
5,1.145,1.610,4.351,9.236,11.070,12.833,15.086,16.750


查表自由度为1,p=0.05的卡方值为3.841，而此比例卡方值10.01 > 3.841，因为此 p < 0.05，说明原假设在0.05的显著性水平下是可以拒绝的。也就是说原假设不成立。即“两种疗效无差别” 这个假设可以拒绝。

**ChiMerge分箱算法**

ChiMerge卡方分箱算法由Kerber于1992提出。   
它主要包括两个阶段：初始化阶段和自底向上的合并阶段。   
- 初始化阶段，首先按照属性值的大小进行排序（对于非连续性，需要先做数值转换，比如转为坏人率，然后排序），然后每个属性值单独作为一组。
- 合并阶段：
    - 对每一对相邻的组，计算卡方值。
    - 根据计算的卡方值，对其中最小的一对邻组合并为一组
    - 不断重复（1），（2），直到计算出的卡方值不低于事先设定的阈值，或者分组数达到一定的条件（如最小分组数5，最大分组数8）。
    
值得注意的是，小编之前发现有的实现方法在合并阶段，计算的并非相邻组的卡方值（只考虑在此两组内的样本，并计算期望频数），因为他们用整体样本来计算此相邻两组的期望频数。

**卡方值计算**

计算卡方值的函数需要输入numpy格式的频数表。对于pandas数据集，只需使用pd.crosstable计算即可，例如变量'总账户数'与目标变量'是否坏客户'的频数表，如下：

In [25]:
import pandas as pd
from pandas import DataFrame,Series
good_count = DataFrame([3,140,341,462,576,699,834,915,1007,1080,1129],columns=['good'],
                    index=range(2,13))
good_count.index.name = 'account_num'
bad_count = DataFrame([1,42,79,91,108,132,172,166,187,199,198],columns=['bad'],
                   index=range(2,13))
bad_count.index.name = 'account_num'
account_table = pd.merge(good_count,bad_count,left_index=True,right_index=True,how='inner')
account_table

,good,bad
account_num,,
2,3,1
3,140,42
4,341,79
5,462,91
6,576,108
7,699,132
8,834,172
9,915,166
10,1007,187


每一行代表一个区间(组)的频数，如上图第一行表示总账户数在[2,3)这个组内对应的好客户3个，坏客户1个。将频数表转成numpy数组，然后调用函数计算卡方值，计算逻辑如下：   
- 计算第i行的总数Ri
- 计算第j列的总数Cj
- 计算总频数N
- 计算第i,j项的期望频数Eij = Ri * Cj / N
- 求的每一项中的卡方(Aij - Eij)\*\*2/Eij
- 由于期望频数Eij有可能是0，此时上一步计算出来的结果无意义，需要清除，不计入最终结果。
- 把所有项的卡方相加得到卡方值   

In [88]:
def chi2_a(arr):
    '''
    计算卡方值
    arr:频数统计表，二维numpy数组
    '''
    assert(arr.ndim==2)
    #计算每行总频数
    R_N = arr.sum(axis=1)
    #计算每列总频数
    C_N = arr.sum(axis=0)
    #总频数
    N = arr.sum()
    # 计算期望频数 C_i * R_j / N
    E = np.ones(arr.shape) * C_N / N
    E = (E.T * R_N).T
    square = (arr-E)**2 /E
    # 期望频数为0时，做除数没有意义，不计入卡方值
    square[E==0] = 0
    #卡方值
    v = square.sum()
    return v

In [89]:
arr = account_table.values
chi2_a(arr)

12.215820314650935

In [90]:
def chi2_b(dataframe,good_col,bad_col):
    df = dataframe.copy()
    sub_total = df[good_col] + df[bad_col]
    good_total = df[good_col].sum()
    bad_total = df[bad_col].sum()
    total = good_total + bad_total
    df['good_experience'] = sub_total * good_total / total
    df['bad_experience'] = sub_total * bad_total / total
    chi2_good = df[['good','good_experience']].apply(lambda x:(x[0]-x[1])**2/x[1],axis=1)
    chi2_bad = df[['bad','bad_experience']].apply(lambda x:(x[0]-x[1])**2/x[1],axis=1)
    return sum(chi2_good) + sum(chi2_bad)

In [91]:
chi2_b(account_table,'good','bad')

12.215820314650935

**ChiMerge分箱算法**

卡方分箱函数可以根据最大分组数目和卡方阈值来控制最终的分箱数。  
如果调用时既没有设置最大分组数，也没有指定阈值，那么函数会自动使用95%的置信度设置阈值。   
分箱逻辑是：
- 初始时，所有变量值都自成一组，统计频数。
- 然后按照各组起始值从小到大，依次扫描，取出两组拼成计算卡方值。如果当前计算出的卡方值小于已观察到的最小卡方值，则标记当前坐标，并更新已观察最小卡方值为当前值。
- 扫描一遍后，如果当前分组数大于最大分组数，或者最小卡方值小于阈值，就将最小卡方值对应的两组频数合并，区间也合并。并回第2步执行。否则，停止合并。输出当前各组的区间切分点。

In [92]:
account_table

,good,bad
account_num,,
2,3,1
3,140,42
4,341,79
5,462,91
6,576,108
7,699,132
8,834,172
9,915,166
10,1007,187


In [ ]:
df2 = df.copy()
df2.groupby([col,target]).count

In [ ]:
def chi_merge_a(df,col,target,max_groups=None,threshold=None):
    '''
    卡方分箱
    df:pandas dataframe数据集
    col:需要分箱的变量名（数值型）
    taget:类标签
    max_groups:最大分组数
    threshold:卡方阈值，如果未指定max_groups,默认使用置信度95%设置threshold
    return:包括各组的起始值的列表
    '''
    freq_tab = pd.crosstab(df[col],df[target])
    
    #转成numpy数组用于计算
    freq = freq_tab.values
    
    #初始分组切分点，每个变量值都是切分点。每组中只包含一个变量值
    #分组区间是左闭右开的，如cutoffs=[1,2,3],则表示区间[1,2)
    #[2,3),[3,3+)
    cutoffs = freq_tab.index.values
    
    #如果没有指定最大分组
    if max_groups is None:
        #如果没有指定卡方阈值，就以95%的置信度(自由度为类数目-1)设定阈值
        if threshold is None:
            cls_num = freq.shape[-1]
            threshold = chi2.isf(0.05,df=cls_num - 1)
    while True:
        minvalue = None
        minidx = None
        #从第1组开始，依次取两组计算卡方值，并判断是否小于当前最小的卡方
        for i in range(len(freq) - 1):
            v = chi2(freq[i:i+2])
            # 小于当前最小卡方，更新最小值
            if minvalue is None or minvalue > v:
                minvalue = v
                minidx = i
        if(max_groups is not None and max_groups < len(freq) \
           or (threshold is not None and minvalue < threshold)):
            # minidx 后一行合并到minidx
            tmp = freq[minidx] + freq[minidx + 1]
            freq[minidx] = tmp
            #删除minidx后一行
            freq = np.delete(freq,minidx+1,0)
            #删除对应的切分点
            cutoffs = np.delete(cutoffs,minidx+1,0)
        else:#最小卡方值不小于阈值，停止合并。
            break
    return cutoffs